In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
import os.path
from time import sleep
import matplotlib.pyplot as plt
import re
import string
import lxml

In [ ]:
df = pd.read_csv('mojoScrape.csv')
testdf = df.head()
testdf

In [19]:
# def getHtml(tail):
#     top = 'http://www.boxofficemojo.com/movies/?id='
#     buildURL = top + tail
#     header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
#     response = requests.get(buildURL, headers = header)
#     sleep(.5)
#     status = response.status_code
#     with (open('html/{}'.format(tail),'w'))as savefile:
#         savefile.write(response.text)
#     return status


# def checkPathOpen(tail):
#     if os.path.isfile('html/{}'.format(tail))==False:
#         getHtml(tail)
#         sleep(.5)
#     with (open('html/{}'.format(tail),'r')) as infile:
#         soup = BeautifulSoup(infile,'lxml')
#         return soup


def getHtml(tail):
    top = 'http://www.boxofficemojo.com/movies/?id='
    buildURL = top + tail
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(buildURL, headers = header)
    sleep(.5)
    status = response.status_code
    with (open('htmlThirdTry/{}'.format(tail),'w'))as savefile:
        savefile.write(response.text)
    return status


def checkPathOpen(tail):
#     if os.path.isfile('htmlThirdTry/{}'.format(tail))==False:
#         getHtml(tail)
#         sleep(.5)
    with (open('htmlThirdTry/{}.html'.format(tail),'r')) as infile:
        soup = BeautifulSoup(infile,'lxml')
        return soup


# df['status'] = df['flatTitle'].map(lambda x: getHtml(x))


In [21]:
# scraping functions

def getDomesticGross(tail):
    soup = checkPathOpen(str(tail))
    dtgRE = re.compile('Domestic Total Gross')
    if soup.find(text=dtgRE):
        dtg = int(soup.find(text=dtgRE).findNextSibling().text.replace('$','').replace(',',''))
        return dtg
    else:
        return np.nan


    
def getProdBudget(tail):
    ret = re.compile('Production Budget')
    soup = checkPathOpen(str(tail))
    obj = soup.find(text=ret)
    if not obj:
        return np.nan
    if obj.findNextSibling():
        sib = obj.findNextSibling()
        if sib.text != 'N/A':
            st = sib.text
            if re.search('\$',st):
                stMdol = st.replace('$','')
                if re.search('.',stMdol):
                    v = stMdol.replace('.','').replace(' million','00000')
                    return v
                elif re.search(' million',stMdol):
                    return int(stMdol.replace(' million','000000'))
                else:
                    return np.nan
            else:
                return np.nan
        else:
            return np.nan
    else:
        return np.nan


    
def getProdBudget(tail):
    es = re.compile('[%s]' % re.escape(string.punctuation))
    ret = re.compile('Production Budget')
    soup = checkPathOpen(str(tail))
    obj = soup.find(text=ret)
    if not obj:
        return np.nan
    if obj.findNextSibling():
        sib = obj.findNextSibling()
        if sib.text != 'N/A':
            st = sib.text
            
            if re.search('\$',st):
                stMdol = st.replace('$','')
                if re.search('.',stMdol):
                    v = stMdol.replace('.','').replace(' million','00000')
                    return int(es.sub('',v))
                elif re.search(' million',stMdol):
                    return int(stMdol.replace(' million','000000'))
                else:
                    return np.nan
            else:
                return np.nan
        else:
            return np.nan
    else:
        return np.nan

    
def getForeignGross(tail):
    fo = re.compile('\+\\xa0Foreign:')
    soup = checkPathOpen(str(tail))
    if not soup.find('div', class_ = 'mp_box_content'):
        return np.nan
    if not soup.find('div', class_ = 'mp_box_content').find('table'):
        return np.nan
    tab = soup.find('div', class_ = 'mp_box_content').find('table').extract()
    rows = tab.find_all('tr')
    data = []
    for row in rows:
        cols =row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    if len(data)==4:
        if re.search(fo,data[1][0]):
            if data[1][1] == 'N/A' or data[1][1] == 'n/a':
                return np.nan
            else:
                d = data[1][1]
                return int(d.replace('$','').replace(',',''))
        else:
            return np.nan
    else: 
        return np.nan
    
    
def getWorldGross(tail):
    d = getDomesticGross(str(tail))
    f = getForeignGross(str(tail))
    if (f != np.nan) and (d != np.nan):
        return d + f
    else:
        return np.nan

    
    

# def getGenre(tail):
#     soup = checkPathOpen(tail)
#     gen = re.compile('Genre')
#     if not soup.find(text=gen):
#         return np.nan
#     else:
#         dtg = soup.find(text=gen).findNextSibling().text.lower()
#         return dtg

    

    

In [14]:
getDomesticGross(58547)

nan

In [ ]:

dfS = pd.read_csv('goodHTML.csv')

In [ ]:
dfS.columns = [['dropMe','flatTitle', 'status']]

In [ ]:
dfS.drop('dropMe', axis=1, inplace=True)

In [ ]:
dfS.head()

In [ ]:

DF = pd.merge(df,dfS,how='left',on='flatTitle')
DF.head(50)

In [ ]:
toScrape = DF[DF['status']== 200]

In [ ]:
toScrape['domesticGross'] = toScrape['flatTitle'].map(lambda x: getDomesticGross(x))
toScrape['prodBudget'] = toScrape['flatTitle'].map(lambda x: getProdBudget(x))

toScrape.head()

In [ ]:
toScrape['foreignGross'] = toScrape['flatTitle'].map(lambda x: getForeignGross(x))

toScrape.head()

In [ ]:
toScrape['worldGross'] = toScrape['flatTitle'].map(lambda x: getWorldGross(x))
toScrape.head()

In [ ]:
toScrape.shape


In [ ]:
toScrape.to_csv('scrapedHTML.csv')


In [ ]:
t = 'http://www.boxofficemojo.com/search/?q=Code%20Name:%20The%20Cleaner'


In [ ]:
thirdTry = pd.to_csv()

In [11]:
thirdTry = pd.read_csv('thirdTry.csv')
thirdTry.head()

,Unnamed: 0,file,size,movieId,title,tmdbId,flatTitle,tail
0,0,$999.html,14316,74486,$9.99,19311,999.htm,/movies/?id=9dot99.htm
1,3,10Years.html,17779,98230,10 Years,58547,10years.htm,/movies/?id=10years.htm
2,5,12.html,31736,56093,12,20714,12.htm,/movies/?id=12.htm
3,7,127Hours.html,17779,81562,127 Hours,44115,127hours.htm,/movies/?id=127hours.htm
4,8,12Rounds.html,14785,70361,12 Rounds,17134,12rounds.htm,/movies/?id=12rounds.htm


In [22]:
thirdTry['domesticGross'] = thirdTry['tmdbId'].map(lambda x: getDomesticGross(x))
thirdTry['prodBudget'] = thirdTry['tmdbId'].map(lambda x: getProdBudget(x))
thirdTry['foreignGross'] = thirdTry['tmdbId'].map(lambda x: getForeignGross(x))
thirdTry['worldGross'] = thirdTry['tmdbId'].map(lambda x: getWorldGross(x))

In [26]:
thirdTry.to_csv('try3.csv')
